In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import cv2
import numpy as np
import os

input_root = "/content/drive/MyDrive/clean_data_MRI/BPD"
output_root = "/content/drive/MyDrive/clean_data_MRI/clean"
os.makedirs(output_root, exist_ok=True)

def process_image(img_path, out_path):
    img = cv2.imread(img_path)
    if img is None:
        return False

    # --- Mask HSV ---
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    lower_yellow = np.array([20, 20, 80])
    upper_yellow = np.array([40, 255, 255])
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    lower_green = np.array([40, 40, 40])
    upper_green = np.array([80, 255, 255])
    mask_green = cv2.inRange(hsv, lower_green, upper_green)

    lower_red1 = np.array([0, 80, 80])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 80, 80])
    upper_red2 = np.array([180, 255, 255])
    mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)

    mask = cv2.bitwise_or(mask_yellow, mask_green)
    mask = cv2.bitwise_or(mask, mask_red)

    # --- Dãn mask ---
    kernel = np.ones((3, 3), np.uint8)
    mask_dilated = cv2.dilate(mask, kernel, iterations=1)

    # --- Chuyển Gray ---
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # --- Inpaint NS ---
    inpainted_gray_ns = cv2.inpaint(gray, mask_dilated, inpaintRadius=3, flags=cv2.INPAINT_NS)

    # --- Median ---
    median_full = cv2.medianBlur(inpainted_gray_ns, 7)

    # --- Chỉ áp dụng median trong vùng mở rộng ---
    kernel_expand = np.ones((35, 35), np.uint8)
    mask_expanded = cv2.dilate(mask_dilated, kernel_expand, iterations=1)

    result_masked = inpainted_gray_ns.copy()
    result_masked[mask_expanded > 0] = median_full[mask_expanded > 0]

    # --- Lưu ảnh ---
    cv2.imwrite(out_path, result_masked)
    return True


# --- Duyệt qua folder con ---
for root, dirs, files in os.walk(input_root):
    # Tạo đường dẫn tương ứng trong output
    rel_path = os.path.relpath(root, input_root)
    out_dir = os.path.join(output_root, rel_path)
    os.makedirs(out_dir, exist_ok=True)

    if files:
        print(f"📂 Đang xử lý folder: {rel_path}")

    for file in files:
        if file.lower().endswith((".png", ".jpg", ".jpeg")):
            in_path = os.path.join(root, file)
            out_name = os.path.splitext(file)[0] + "_median.png"
            out_path = os.path.join(out_dir, out_name)
            process_image(in_path, out_path)


📂 Đang xử lý folder: BPD dau dai
📂 Đang xử lý folder: BPD binh thuong tu 14 den 2808
📂 Đang xử lý folder: BPD nang dam roi mach mac nao that ben
📂 Đang xử lý folder: BPD nao truoc khong phan chia
📂 Đang xử lý folder: BPD xuat huyet noi so
📂 Đang xử lý folder: BPD canxi hoa doi thi
📂 Đang xử lý folder: BPD thung lung sylvius khong tuong xung tuoi thai
📂 Đang xử lý folder: BPD dan ho sau
📂 Đang xử lý folder: BPD dan nao that
📂 Đang xử lý folder: BPD bat san the chai
